## Forecasting SSH with the qsgw-DI model
(Jupyter Notebook prepared by Redouane Lguensat & Clément Ubelmann, 2018)

In [ ]:
import sys
print (sys.version)

In [ ]:
import sys,os,shutil
import numpy
import matplotlib.pylab as plt
import time
import netCDF4 as nc
import qgsw

In [ ]:
file='NATL60OSMO-CJM165_y2012m06d14-y2013m10d01.1d_SSHdegrad.nc'
fid = nc.Dataset(file)
lon1d=numpy.array(fid.variables["nav_lon"][:,:]).squeeze()
lat1d=numpy.array(fid.variables["nav_lat"][:,:]).squeeze()
#lon,lat=numpy.meshgrid(lon1d,lat1d)
SSH=numpy.array(fid.variables["degraded_sossheig"][270,:,:]).squeeze()
print lon1d.shape,lat1d.shape,SSH.shape

In [ ]:
# Set constant Rossby first baroclinic phase speed to constant value
c=SSH*0. + 2.5 # in m/s

tint=86400*30 # Time of propagator integration in seconds. Can be positive or negative
deltat=86400*1 # Time period of outputs
dt=600 # propagator time step

## Applying the QGSW operator

In [ ]:
Hf,trash=qgsw.qgsw(Hi=SSH, c=c, lon=lon1d, lat=lat1d, tint=tint, dtout=deltat, dt=dt,rappel=None,snu=0.)

## Saving the result

In [ ]:
for it in range(numpy.shape(Hf)[0]):
    print it
    ncout = nc.Dataset('testdata1/ssh9_'+str(it)+'.nc', 'w', format='NETCDF3_CLASSIC')
    ncout.createDimension('x', SSH.shape[0])
    ncout.createDimension('y', SSH.shape[1])
    ncout.createDimension('time', None)
    nclon = ncout.createVariable('lon', 'f', ('x','y',))
    nclat = ncout.createVariable('lat', 'f', ('x','y',))
    nctim = ncout.createVariable('time', 'f', ('time',))
#   nchei = ncout.createVariable('SSH', 'f', ('time','y','x',))
    nchei = ncout.createVariable('SSH', 'f', ('x','y',))
    nclon[:] = lon1d[:]
    nclat[:] = lat1d[:]
#nctim[:]=range(0,tint+deltat,deltat)
    nctim[:]=deltat*it
#nchei[:,:,:] = Hf[:,:,:]
    nchei[:,:] = Hf[it,:,:]
    ncout.close()

## Plot

In [ ]:
plt.figure(figsize=(10, 10))
vmin=-0.3
vmax=0.1

plt.subplot(211)
plt.pcolormesh(lon1d,lat1d,SSH, vmin=vmin, vmax=vmax)
plt.colorbar(extend='both', fraction=0.042, pad=0.04)
plt.title('SSH_{t}');

plt.subplot(212)
#plt.pcolormesh(lon,lat,Hf[-1,:,:], vmin=vmin, vmax=vmax)
plt.pcolormesh(lon1d,lat1d,Hf[0,:,:]-SSH)
plt.colorbar(extend='both', fraction=0.042, pad=0.04)
plt.title('SSH_qg_{t+tint} (here after 30 days)');

plt.show()